# 特徴量テーブル閲覧ノートブック v2（自動ルート検出）

このノートブックは、**どのディレクトリから起動しても** プロジェクトルートを自動検出し、
`src.pipelines.loto_pipeline` を確実に import できるように改良されています。

- 生成対象: `artifacts/features_hist.csv`, `features_futr.csv`, `features_stat.csv`
- 未生成の場合は、同梱のサンプル（`examples/sample_loto.csv`）から自動実行します。


In [ ]:
# ルート自動検出（上位へ遡り、'src' と 'config/pipeline_config.yaml' がある場所を探す）
from pathlib import Path
import sys

def find_project_root(start: Path, max_up: int = 6):
    p = start.resolve()
    for _ in range(max_up + 1):
        if (p / "src").is_dir() and (p / "config" / "pipeline_config.yaml").exists():
            return p
        p = p.parent
    return None

# Jupyter の CWD を起点に検索
CWD = Path.cwd()
ROOT = find_project_root(CWD)

if ROOT is None:
    # ノートブックが同梱された配布物想定のフォールバック
    # （このノートブックの親→親をルート候補とする）
    ROOT = Path(".").resolve().parent

print("Detected ROOT:", ROOT)

# import パスに追加（'src' を含むディレクトリを追加）
if str(ROOT) not in sys.path:
    sys.path.insert(0, str(ROOT))

In [ ]:
# 主要パス
from pathlib import Path

BASE = ROOT
ART = BASE / 'artifacts'
CFG = BASE / 'config' / 'pipeline_config.yaml'
SAMPLE = BASE / 'examples' / 'sample_loto.csv'

HIST = ART / 'features_hist.csv'
FUTR = ART / 'features_futr.csv'
STAT = ART / 'features_stat.csv'

print("BASE:", BASE)
print("ART:", ART)
print("CFG:", CFG)
print("SAMPLE:", SAMPLE)

In [ ]:
# 必要に応じてパイプラインを実行（サンプル）
run_if_missing = True

def _maybe_run_pipeline():
    try:
        from src.pipelines.loto_pipeline import run as run_pipeline
    except Exception as e:
        print("パイプラインのimportに失敗:", e)
        # デバッグ情報
        import sys, os
        print("sys.path[0:5]:", sys.path[:5])
        print("ディレクトリ一覧（ROOT直下）:", os.listdir(str(BASE)))
        return
    ART.mkdir(parents=True, exist_ok=True)
    print("パイプライン実行中...")
    res = run_pipeline(str(SAMPLE), str(ART), str(CFG))
    print("生成完了:", res)

if run_if_missing and not (HIST.exists() and FUTR.exists() and STAT.exists()):
    _maybe_run_pipeline()
else:
    print("features_*.csv が見つかりました（または実行スキップ）。")

In [ ]:
# 読み込み＆プレビュー
import pandas as pd

def _safe_read_csv(p: Path):
    if not p.exists():
        print(f"ないよ: {p}")
        return None
    try:
        return pd.read_csv(p)
    except Exception as e:
        print(f"読込失敗: {p} -> {e}")
        return None

df_hist = _safe_read_csv(HIST)
df_futr = _safe_read_csv(FUTR)
df_stat = _safe_read_csv(STAT)

for name, df in [("hist", df_hist), ("futr", df_futr), ("stat", df_stat)]:
    if df is not None:
        print(f"{name}: shape={df.shape}")
        display(df.head(10))

In [ ]:
# フィルタ/結合/プロファイル/可視化
import pandas as pd
import matplotlib.pyplot as plt

def preview_hist(unique_id=None, start=None, end=None, cols=None, head=20):
    if df_hist is None:
        print("df_hist がありません")
        return None
    out = df_hist.copy()
    if unique_id is not None:
        out = out[out['unique_id'] == unique_id]
    if start is not None:
        out = out[out['ds'] >= str(start)]
    if end is not None:
        out = out[out['ds'] <= str(end)]
    if cols is not None:
        keep = ['unique_id','ds'] + [c for c in cols if c in out.columns]
        out = out[keep]
    display(out.head(head))
    return out

def merged_view(limit=50):
    if (df_hist is None) or (df_futr is None) or (df_stat is None):
        print("いずれかのテーブルがありません")
        return None
    m = df_hist.merge(df_futr, on=['unique_id','ds'], how='left')\
               .merge(df_stat, on=['unique_id'], how='left')
    display(m.head(limit))
    return m

def profile(df, name):
    if df is None:
        return
    print(f"[{name}] shape={df.shape}")
    display(pd.DataFrame({
        "dtype": df.dtypes.astype(str),
        "n_unique": df.nunique(),
        "n_missing": df.isna().sum(),
    }))

def plot_series(unique_id='loto6', col='y', head=None):
    if df_hist is None:
        print("df_hist がありません")
        return
    d = df_hist[df_hist['unique_id'] == unique_id].copy()
    if head is not None:
        d = d.head(head)
    try:
        d['ds'] = pd.to_datetime(d['ds'])
    except Exception:
        pass
    d = d.sort_values('ds')
    if col not in d.columns:
        print(f"列がありません: {col}")
        return
    plt.figure()
    plt.plot(d['ds'], d[col])
    plt.title(f"{unique_id} - {col}")
    plt.xlabel("ds")
    plt.ylabel(col)
    plt.tight_layout()
    plt.show()

# 例
_ = preview_hist(unique_id='loto6', head=20)
_ = merged_view(30)
profile(df_hist, "hist")
profile(df_futr, "futr")
profile(df_stat, "stat")
